## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf   

In [0]:
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/gdrive/My Drive/AIML/prices.csv')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Check all columns in the dataset

In [7]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [8]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
subset = pd.read_csv('/prices.csv',nrows=1000)


In [12]:
print(subset.head())

                  date symbol        open  ...         low        high     volume
0  2016-01-05 00:00:00   WLTW  123.430000  ...  122.309998  126.250000  2163600.0
1  2016-01-06 00:00:00   WLTW  125.239998  ...  119.940002  125.540001  2386400.0
2  2016-01-07 00:00:00   WLTW  116.379997  ...  114.930000  119.739998  2489500.0
3  2016-01-08 00:00:00   WLTW  115.480003  ...  113.500000  117.440002  2006300.0
4  2016-01-11 00:00:00   WLTW  117.010002  ...  114.089996  117.330002  1408600.0

[5 rows x 7 columns]


In [0]:
data_sample=subset.drop(['date','symbol'],axis=1)

In [63]:
data_sample.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
X=data_sample.drop(['volume'],axis=1)

In [0]:
y=data_sample['volume']

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
from sklearn.preprocessing import Normalizer
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
transformer = Normalizer()
X_train= transformer.fit_transform(X_train)

In [20]:
X_train.shape

(700, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [78]:
w

<tf.Tensor 'sub_5:0' shape=(4, 1) dtype=float32>

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b]) # this function returns the derivative of loss function
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
y_train=np.array(y_train)


In [25]:
X_train.shape

(700, 4)

In [24]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)))

Current Loss on iteration 0 tf.Tensor(223490320000000.0, shape=(), dtype=float32)
Current Loss on iteration 1 tf.Tensor(221288360000000.0, shape=(), dtype=float32)
Current Loss on iteration 2 tf.Tensor(219259040000000.0, shape=(), dtype=float32)
Current Loss on iteration 3 tf.Tensor(217388830000000.0, shape=(), dtype=float32)
Current Loss on iteration 4 tf.Tensor(215665200000000.0, shape=(), dtype=float32)
Current Loss on iteration 5 tf.Tensor(214076720000000.0, shape=(), dtype=float32)
Current Loss on iteration 6 tf.Tensor(212612780000000.0, shape=(), dtype=float32)
Current Loss on iteration 7 tf.Tensor(211263640000000.0, shape=(), dtype=float32)
Current Loss on iteration 8 tf.Tensor(210020230000000.0, shape=(), dtype=float32)
Current Loss on iteration 9 tf.Tensor(208874300000000.0, shape=(), dtype=float32)
Current Loss on iteration 10 tf.Tensor(207818220000000.0, shape=(), dtype=float32)
Current Loss on iteration 11 tf.Tensor(206844920000000.0, shape=(), dtype=float32)
Current Loss o

### Get the shapes and values of W and b

In [25]:
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1354918.1]
 [1359816.4]
 [1341065.8]
 [1371394.8]]
Bias:
 [2713755.]


In [31]:
files.download('R6_Internal_Lab_UpdatedTF2_Prices_Iris-3.h5')
from google.colab import files

files.download('R6_Internal_Lab_UpdatedTF2_Prices_Iris-3.h5')

NameError: ignored

### Model Prediction on 1st Examples in Test Dataset

In [26]:
X_test=transformer.fit_transform(X_test)
prediction(X_test[0:1],w,b).numpy()

array([[5427437.]], dtype=float32)

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
df_iris=pd.read_csv('/11_Iris.csv')

In [0]:
df1_iris=df_iris

In [52]:
df1_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
!pip install -q sklearn

In [54]:
df1_iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [0]:
dummies=pd.get_dummies(df_iris['Species'])

In [0]:
df = pd.concat([df_iris, dummies], axis=1) 

In [0]:
df.drop('Species',axis=1,inplace=True)

In [59]:
df.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],
      dtype='object')

### Splitting the data into feature set and target set

In [0]:
X=df.drop(columns=['Iris-setosa','Iris-versicolor','Iris-virginica'])

In [0]:
y=df[['Iris-setosa','Iris-versicolor','Iris-virginica']]

In [0]:
trainX,testX,trainY,testY=train_test_split(X,y,test_size=0.2,random_state=1)

In [64]:
testX.drop('Id',axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()



#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3,input_shape=(4,), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
trainX.drop('Id',axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [79]:
trainX.shape

(120, 4)

In [77]:
testY.shape

(30, 3)

In [71]:
testX.drop('Id',axis=1,inplace=True)

KeyError: ignored

### Model Training 

In [81]:
model.fit(trainX.values, trainY.values, 
          validation_data=(testX.values, testY.values), 
          epochs=10,
          batch_size=32)   

Train on 120 samples, validate on 30 samples
Epoch 1/10
120/120 [==============================] - 0s 3ms/sample - loss: 1.8661 - acc: 0.2000 - val_loss: 4.2895 - val_acc: 0.2000
Epoch 2/10
120/120 [==============================] - 0s 184us/sample - loss: 1.7977 - acc: 0.2333 - val_loss: 4.1543 - val_acc: 0.2000
Epoch 3/10
120/120 [==============================] - 0s 191us/sample - loss: 1.6927 - acc: 0.2667 - val_loss: 4.0379 - val_acc: 0.2000
Epoch 4/10
120/120 [==============================] - 0s 172us/sample - loss: 1.6095 - acc: 0.2583 - val_loss: 3.9259 - val_acc: 0.2000
Epoch 5/10
120/120 [==============================] - 0s 199us/sample - loss: 1.5444 - acc: 0.2833 - val_loss: 3.8183 - val_acc: 0.2000
Epoch 6/10
120/120 [==============================] - 0s 188us/sample - loss: 1.5010 - acc: 0.2833 - val_loss: 3.7160 - val_acc: 0.2333
Epoch 7/10
120/120 [==============================] - 0s 180us/sample - loss: 1.4414 - acc: 0.3083 - val_loss: 3.6177 - val_acc: 0.2333
Epoch

### Model Prediction

In [82]:
model.predict(testX.values)

array([[9.7728676e-05, 9.0542835e-01, 9.4473958e-02],
       [3.4802903e-03, 5.4517800e-01, 4.5134172e-01],
       [4.6687474e-04, 4.5309678e-01, 5.4643637e-01],
       [2.1884871e-04, 8.7349653e-01, 1.2628461e-01],
       [6.1173087e-05, 2.3346515e-01, 7.6647365e-01],
       [5.3532352e-04, 3.5336021e-01, 6.4610445e-01],
       [4.1568067e-04, 3.2882926e-01, 6.7075509e-01],
       [4.9014419e-04, 7.7990752e-01, 2.1960236e-01],
       [1.3184268e-03, 7.8362644e-01, 2.1505514e-01],
       [3.7557268e-04, 2.2998743e-01, 7.6963705e-01],
       [1.9706490e-03, 3.6313674e-01, 6.3489264e-01],
       [2.6545761e-04, 8.3179712e-01, 1.6793743e-01],
       [3.4753667e-04, 2.2149739e-01, 7.7815503e-01],
       [5.0354184e-04, 4.0657201e-01, 5.9292448e-01],
       [1.0947842e-03, 3.5211727e-01, 6.4678794e-01],
       [1.7560886e-03, 7.9819363e-01, 2.0005019e-01],
       [1.6418329e-03, 4.2992672e-01, 5.6843144e-01],
       [1.5588307e-03, 3.0632547e-01, 6.9211566e-01],
       [4.1026223e-04, 8.500

### Save the Model

In [0]:
model.save('R6_Internal_Lab_UpdatedTF2_Prices_Iris-3_sol.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?